# Storage Systems and Demand-Side Management

This tutorial covers how to model storage systems and demand-side management in PyPSA. We'll explore different types of storage, their characteristics, and how to optimize their operation.

## Types of Storage

PyPSA supports various types of storage systems:

1. **Electrical Storage** (e.g., batteries)
2. **Thermal Storage** (e.g., heat storage)
3. **Hydro Storage** (e.g., pumped hydro)
4. **Chemical Storage** (e.g., hydrogen)

Let's create a network with different storage types:

In [ ]:
import pypsa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Create network with time series
network = pypsa.Network()
snapshots = pd.date_range("2024-01-01", periods=24, freq="h")
network.set_snapshots(snapshots)

# Add buses
network.add("Bus", "electricity", v_nom=20.0)
network.add("Bus", "heat", carrier="heat")
network.add("Bus", "hydrogen", carrier="hydrogen")

# Add carriers
network.add("Carrier", "gas", co2_emissions=0.2)
network.add("Carrier", "solar")

## Adding Storage Components

Let's add different types of storage systems:

In [ ]:
# Add generators
network.add(
    "Generator",
    "solar",
    bus="electricity",
    p_nom=100, # Fixed nominal power capacity (MW)
    marginal_cost=0, # Assume zero marginal cost for solar
    p_max_pu=pd.Series(
        [0.0, 0.0, 0.0, 0.0, 0.1, 0.3, 0.5, 0.7, 0.8, 0.9, 0.9, 0.8,
         0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0],
        index=network.snapshots
    ),
    carrier="solar"
)

# Add a backup gas generator to ensure feasibility
network.add(
    "Generator",
    "gas_backup",
    bus="electricity",
    p_nom=100, # Sufficient capacity to cover peak load
    marginal_cost=50, # Cost per MWh electrical
    carrier="gas",
    efficiency=0.4 # Fuel efficiency
)

# Add battery storage (Fixed capacity for operational analysis)
network.add(
    "Store",
    "battery",
    bus="electricity",
    e_nom=100,  # Fixed Energy capacity (MWh)
    p_nom=20,  # Fixed Power capacity (MW)
    e_nom_extendable=False, # Focus on operation, not investment
    p_nom_extendable=False,
    e_cyclic=True,  # Allow cyclic operation within snapshots
    e_min_pu=0.1,  # Minimum state of charge (10%)
    e_max_pu=0.9,  # Maximum state of charge (90%) - avoid full charge/discharge
    standing_loss=0.001, # Small self-discharge rate (0.1% per hour)
    efficiency_store=0.95, # Charging efficiency
    efficiency_dispatch=0.95, # Discharging efficiency
    marginal_cost=0.1  # Small operating cost per MWh cycled (charge OR discharge)
    # capital_cost_e=100,  # Removed for operational focus
    # capital_cost_p=1000,  # Removed for operational focus
)

# Add heat storage (Fixed capacity)
network.add(
    "Store",
    "heat_storage",
    bus="heat",
    e_nom=200,
    p_nom=40,
    e_nom_extendable=False,
    p_nom_extendable=False,
    e_cyclic=True,
    e_min_pu=0.1,
    e_max_pu=0.9,
    standing_loss=0.01,  # Heat loss (1% per hour)
    efficiency_store=0.98, # High efficiency for thermal storage
    efficiency_dispatch=0.98,
    marginal_cost=0.01 # Very low operational cost
    # capital_cost_e=50,
    # capital_cost_p=500,
)

# Add hydrogen storage (Fixed capacity)
network.add(
    "Store",
    "hydrogen_storage",
    bus="hydrogen",
    e_nom=1000, # Energy capacity in MWh_H2 (LHV)
    p_nom=100, # Power capacity (MW_H2)
    e_nom_extendable=False,
    p_nom_extendable=False,
    e_cyclic=True,
    e_min_pu=0.1,
    e_max_pu=0.9,
    standing_loss=0.001,  # Leakage (0.1% per hour)
    efficiency_store=0.99, # Very high storage efficiency itself
    efficiency_dispatch=0.99,
    marginal_cost=0.05 # Small operational cost
    # capital_cost_e=200,
    # capital_cost_p=2000,
)

## Adding Power-to-X and X-to-Power Components

Let's add components to convert between different energy carriers:

In [ ]:
# Add electrolyzer (Power-to-Hydrogen) - Fixed Capacity
network.add(
    "Link",
    "electrolyzer",
    bus0="electricity", # Input bus
    bus1="hydrogen", # Output bus
    p_nom=50, # Fixed power capacity (MW electrical input)
    p_nom_extendable=False,
    efficiency=0.7, # Efficiency: MWh_H2 (LHV) output per MWh_el input
    marginal_cost=1 # Small operational cost per MWh_el input
    # capital_cost=1000 # Removed for operational focus
)

# Add fuel cell (Hydrogen-to-Power) - Fixed Capacity
network.add(
    "Link",
    "fuel_cell",
    bus0="hydrogen", # Input bus
    bus1="electricity", # Output bus
    p_nom=50, # Fixed power capacity (MW H2 input)
    p_nom_extendable=False,
    efficiency=0.6, # Efficiency: MWh_el output per MWh_H2 (LHV) input
    marginal_cost=2 # Small operational cost per MWh_H2 input
    # capital_cost=1000
)

# Add heat pump (Power-to-Heat) - Fixed Capacity
network.add(
    "Link",
    "heat_pump",
    bus0="electricity", # Input bus
    bus1="heat", # Output bus
    p_nom=30, # Fixed power capacity (MW electrical input)
    p_nom_extendable=False,
    efficiency=3.0,  # Coefficient of Performance (COP): MWh_heat output per MWh_el input
    marginal_cost=0.5 # Small operational cost per MWh_el input
    # capital_cost=800
)

## Adding Loads and Demand-Side Management

Let's add loads with different flexibility options:

In [ ]:
# Define simple daily load profiles
# Electricity: Higher in evening
base_elec_load = np.array([0.6, 0.5, 0.5, 0.5, 0.6, 0.7, 0.8, 0.8, 0.8, 0.7, 0.7, 0.7,
                          0.7, 0.7, 0.8, 0.9, 1.0, 1.0, 0.9, 0.8, 0.7, 0.6, 0.6, 0.6])
# Heat: Higher in morning and evening
base_heat_load = np.array([0.8, 0.7, 0.6, 0.6, 0.7, 0.9, 1.0, 0.9, 0.8, 0.7, 0.6, 0.6,
                          0.6, 0.6, 0.7, 0.8, 0.9, 1.0, 1.0, 0.9, 0.8, 0.8, 0.8, 0.8])
# Hydrogen: Constant base, representing industrial demand maybe
base_h2_load = np.ones(24) * 0.8

# Create pandas Series with the snapshots index
elec_load_profile = pd.Series(base_elec_load * 50, index=network.snapshots) # Peak of 50 MW
heat_load_profile = pd.Series(base_heat_load * 30, index=network.snapshots) # Peak of 30 MWth
h2_load_profile = pd.Series(base_h2_load * 20, index=network.snapshots)   # Avg of 16 MW_H2

# Add fixed (but time-varying) electricity load
network.add(
    "Load",
    "electricity_load",
    bus="electricity",
    p_set=elec_load_profile # Use the defined profile
)

# Add flexible heat load (DSM)
network.add(
    "Load",
    "heat_load",
    bus="heat",
    p_set=heat_load_profile, # Use the defined profile
    p_max_pu=1.2,  # Can consume up to 120% of its profile value in any hour
    p_min_pu=0.8   # Must consume at least 80% of its profile value in any hour
    # Note: For true DSM (shifting), you might use Storage elements
    # or more complex constraints, but p_max/min_pu shows simple flexibility.
)

# Add flexible hydrogen load (DSM)
network.add(
    "Load",
    "hydrogen_load",
    bus="hydrogen",
    p_set=h2_load_profile, # Use the defined profile
    p_max_pu=1.5,  # Can consume up to 150%
    p_min_pu=0.5   # Must consume at least 50%
)

## Optimizing Storage Operation

Let's solve the optimization problem and analyze the results:

In [ ]:
# Solve the linear optimal power flow (dispatch)
# Initialize status variables in case the try block fails early
status = None
condition = None

print("Attempting to solve optimization...")
try:
    status, condition = network.optimize()

    print(f"Solver Status: {status}, Termination Condition: {condition}")

except Exception as e:
    print(f"An error occurred during optimization: {e}")

# Proceed with results analysis only if optimization was successful
if status == "ok" and condition == "optimal":
    print(f"Optimization successful. Total operational cost: {network.objective:.2f}")

    try:
        # Plot storage operation
        plt.figure(figsize=(12, 6))
        network.stores_t.e.plot()
        plt.title('Storage Energy Levels Over Time')
        plt.xlabel('Time')
        plt.ylabel('Energy (MWh)')
        plt.legend(title='Storage Unit')
        plt.grid(True)
        plt.tight_layout()
        plt.show()

        # Plot storage power dispatch (positive = dispatch/discharge, negative = store/charge)
        plt.figure(figsize=(12, 6))
        network.stores_t.p.plot()
        plt.title('Storage Power Dispatch Over Time')
        plt.xlabel('Time')
        plt.ylabel('Power (MW)')
        plt.legend(title='Storage Unit')
        plt.grid(True)
        plt.tight_layout()
        plt.show()

        # Plot power flows for conversion links (p0 = input power)
        plt.figure(figsize=(12, 6))
        network.links_t.p0.plot()
        plt.title('Power Input to Conversion Components (Links)')
        plt.xlabel('Time')
        plt.ylabel('Power (MW)')
        plt.legend(title='Component (Link)')
        plt.grid(True)
        plt.tight_layout()
        plt.show()

        # Plot flexible load dispatch vs setpoint
        plt.figure(figsize=(12, 6))
        flexible_loads = network.loads[network.loads.p_max_pu != 1].index
        if not flexible_loads.empty:
            # Plot each flexible load separately to avoid list labels
            for load in flexible_loads:
                network.loads_t.p[load].plot(style='-', label=f'{load} Dispatch')

                # For setpoint, check if it's a time series or static value
                if isinstance(network.loads.p_set[load], pd.Series):
                    network.loads.p_set[load].plot(style=':', label=f'{load} Setpoint')
                else:
                    # If static value, create a constant series to plot
                    pd.Series(
                        network.loads.p_set[load],
                        index=network.snapshots
                    ).plot(style=':', label=f'{load} Setpoint')

            plt.title('Flexible Load Dispatch vs. Setpoint Profile')
            plt.xlabel('Time')
            plt.ylabel('Power (MW)')
            plt.legend()
            plt.grid(True)
            plt.tight_layout()
            plt.show()
        else:
            print("No flexible loads found to plot.")

    except Exception as e:
        print(f"An error occurred during plotting: {e}")

elif status is not None:
    print("Skipping results analysis due to non-optimal solver status.")
else:
    print("Skipping results analysis because optimization did not run or failed before status check.")

## Key Storage and Balancing Concepts

1. **Storage Characteristics**
   - Energy capacity (e_nom)
   - Power capacity (p_nom)
   - Efficiency (charging/discharging)
   - Standing losses
   - Minimum energy levels

2. **Power-to-X Technologies**
   - Convert electricity to other carriers
   - Examples: electrolyzers, heat pumps
   - Efficiency losses in conversion

3. **Demand-Side Management**
   - Flexible loads
   - Time-shifting capability
   - Maximum/minimum power limits

4. **Storage Operation**
   - Charging/discharging patterns
   - Energy level variations
   - Integration with renewables

## Next Steps

In the next tutorial, we'll explore sector coupling in more detail, including the integration of power, heat, and transport sectors.